# LLaMA2-7b 모델을 KoALPACA 모델로의 학습 후 평가

## 1. 개요
* 모델명 :[meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) 모델을 아래의 QLoRA로 SFT 한 모델
* 데이터셋
    * 한국어 Alpaca Dataset : [ko_alpaca_data.json](https://github.com/Beomi/KoAlpaca/blob/main/ko_alpaca_data.json)
    * 네이버 지식인 베스트 데이터 : [KoAlpaca_v1.1.json](https://raw.githubusercontent.com/Beomi/KoAlpaca/main/KoAlpaca_v1.1.jsonl)

In [1]:
import os
import torch
from torch.utils.data import Dataset

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel,
)
import transformers
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    GenerationConfig,
)

import pandas as pd
import json

## 2. Set Arguments

### 2.1 base 관련 파라미터

In [2]:
CHECKPOINT_DIR = '/workspace/llama2-KoAlpaca-Finetuning/output/checkpoint-2200'

In [3]:
BASE_PATH = '/workspace/llama2-KoAlpaca-Finetuning'
RANDOM_SEED = 777
HUGGINGFACE_TOKEN = ''

In [4]:
CACHE_DIR=os.path.join('/workspace', ".cache") 

### 2.2 Model 관련 파라미터

In [5]:
MODEL_NAME_OR_PATH = 'meta-llama/Llama-2-7b-hf'
TORCH_DTYPE=torch.float16

### 2.3 양자화 관련 파라미터

In [6]:
LOAD_IN_4BIT=True                                   # Enable 4-bit quantization
BNB_4BIT_QUANT_TYPE="nf4"                           # BNB 4-bit quantization type
BNB_4BIT_COMPUTE_DTYPE=torch.bfloat16               # BNB 4-bit compute dtype
BNB_4BIT_USE_DOUBLE_QUANT=True                      # BNB 4-bit use double quantization

### 2.4 LoRA 관련 파라미터

In [7]:
R=8                                                 # Lora attention dimension
LORA_ALPHA=16                                       # Lora alpha parameter
LORA_DROPOUT=0.05                                   # Lora dropout probability
FAN_IN_FAN_OUT=False                                # Lora fan in fan out
BIAS="none"                                         # Lora bias type
TARGET_MODULES=["q_proj", "v_proj"]                 # Lora target modules
INFERENCE_MODE=False                                # Inference mode
TASK_TYPE="CAUSAL_LM"                               # Task type

### 2.5 Tokenizer 관련 파라미터

In [8]:
MAX_LENGTH=512                                     # Max sequence length for tokenizer
TRUNCATION=True                                     # Enable/disable truncation
RETURN_OVERFLOWING_TOKENS=True                      # Return overflowing tokens info
RETURN_LENGTH=True                                  # Return length of encoded inputs
PADDING=True                                        # Enable padding to max sequence length
PADDING_SIDE="right"                                # The side on which the model should have padding appliedㅠ

### 2.6 PROMPT 관련

In [9]:
PROMPT_DICT = {
    "prompt_input": (
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n"
        "명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n\n"
        
        "### Instruction(명령어):%s\n"
        "### Input(입력):%s\n"
        "### Response(응답):"
    ),
    "prompt_no_input": (
        "아래는 작업을 설명하는 명령어입니다.\n"
        "명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n"
        
        "### Instruction(명령어):%s\n"
        "### Response(응답):"
    ),
}

### 2.7 Evaluate 관련

* 모델 Generation 결과 데이터 직렬화 관련
    * 모델 추론 시 GPU 자원 사용 및 추론 시간이 너무 오래걸려서 결과 데이터를 직렬화

In [10]:
EVALUATE_DIR = os.path.join(BASE_PATH, 'evaluate', 'pickles')
GENERATION_DATA_LIST_PICKLE_NAME = 'sh_llama2-alpaca_1st_1000_output.pkl'
GENERATION_DATA_LIST_PICKLE_PATH = os.path.join(EVALUATE_DIR, GENERATION_DATA_LIST_PICKLE_NAME)

* 모델 평가 결과 파일로 저장 관련 

In [11]:
EVALUATION_RESULT_DIR = os.path.join(BASE_PATH, 'evaluate', 'metrics')
EVALUATION_RESULT_FILENAME = os.path.join(EVALUATION_RESULT_DIR, 'sh_llama2-alpaca_1th_evaluation.json')

## 3. 데이터셋 살펴보기 및 하나로 합치기

### 3.1 한국어 Aplaca 데이터셋 살펴보기

In [12]:
DATA_PATH = os.path.join(BASE_PATH, 'datas')
DATASET_KO_ALPACA_FINAL_PATH = os.path.join(DATA_PATH, 'final_ko_alpaca_data.json')

In [13]:
KO_ALPACA_FINAL_PATH = os.path.join(BASE_PATH, DATASET_KO_ALPACA_FINAL_PATH)

with open(KO_ALPACA_FINAL_PATH, 'r', encoding='utf-8') as file:
    ko_alpaca_final = json.load(file)

df_ko_alpaca_final = pd.DataFrame(ko_alpaca_final)
df_ko_alpaca_final.head(5)

,instruction,input,output
0,이메일에서 스팸을 탐지하기 위한 기계 학습 알고리즘을 만드세요.,,이메일 스팸 탐지 알고리즘을 위한 몇 가지 인기있는 기계 학습 모델에는 Suppor...
1,고기 요리 시 맛술을 사용하는 이유와 대체재는 무엇인가요? 어떤 음식에 맛술을 사용...,,안녕하세요! 고기 요리 시 맛술을 사용하는 이유는 냄새와 맛의 개선입니다. 근육 안...
2,대화를 시작하기 위해 트윗을 작성합니다.,,'안녕하세요! 오늘 날씨가 참 좋네요!' (트윗 예시)
3,저녁과 밤의 구분 기준은 무엇인가요? 저녁 8시가 맞나요? 밤 8시가 맞나요?,,저녁과 밤을 구분하는 기준은 명확히 정의되어 있지 않습니다. '저녁'은 해가 지고 ...
4,이 문장의 시작 부분에 전환 단어를 삽입하세요.,그는 오랫동안 수색한 끝에 보물을 찾았습니다.,"그러나, 그는 오랫동안 수색한 끝에 보물을 찾았습니다."


### 3.2 평가를 위해 한국어 Aplaca 데이터셋 1000개의 샘플 뽑기

In [14]:
df_ko_alpaca_final_sampled = df_ko_alpaca_final.sample(1000, random_state=RANDOM_SEED)
df_ko_alpaca_final_sampled

,instruction,input,output
17666,고객 서비스 봇에 가장 적합한 어조를 파악하세요.,,고객 서비스에서 호칭과 제품 또는 서비스에 대한 지식을 바탕으로 열정적인 어조를 사...
69026,동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.,,동물학자들이 말하는 대규모 멸종 사건이 상상이상으로 빨리 벌어지고 있습니다.
20869,예산 수립을 위한 계획 만들기,,"예산 수립 계획을 만들 때는 다음과 같은 단계를 따릅니다. 예산 오브젝티브 설정, ..."
68447,"""Dear John""이라는 표현의 어원은 무엇일까요? 왜 이별 편지나 절교장을 의미...",,"""Dear John letter""는 이별 편지나 절교장을 뜻하는 표현으로 사용됩니다..."
15376,피보나치 수열의 처음 10개의 숫자를 출력하는 함수를 Java로 작성하십시오.,,```java\npublic class Fibonacci {\n\n public ...
...,...,...,...
7944,지구 온난화로 인해 발생할 수 있는 다섯 가지 문제를 브레인스토밍하십시오.,,"지구온난화로 인해 발생할 수 있는 다섯 가지 문제는 먹을 것 부족, 극심한 기후변화..."
1741,"""별이 태어났다""라는 문구를 다른 두 가지 방법으로 다시 쓰십시오.",,"New star is born, A star has arisen"
61640,"다음 기사의 헤드라인을 작성합니다: ""스타틴이 심장 마비 위험을 줄인다는 연구 결과...",,"""연구 결과, 스타틴은 심장 마비 위험을 줄인다."""
43819,해변의 자연물을 사용하여 게임을 디자인하세요.,,"해변에서 찾은 조개껍질을 이용하여, 조개찾기 대회를 개최해보세요."


### 3.2 평가를 위해 한국어 Aplaca 데이터셋 1000개의 샘플 뽑기

In [15]:
formatted_prompts = []
for _, row in df_ko_alpaca_final_sampled.iterrows():
    instruction = row['instruction']
    input_text = row['input']
    if input_text:
        formatted_prompts.append(PROMPT_DICT['prompt_input'] % (instruction, input_text))
    else:
        formatted_prompts.append(PROMPT_DICT['prompt_no_input'] % instruction)

In [16]:
formatted_prompts[:2]

['아래는 작업을 설명하는 명령어입니다.\n명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n### Instruction(명령어):고객 서비스 봇에 가장 적합한 어조를 파악하세요.\n### Response(응답):',
 '아래는 작업을 설명하는 명령어입니다.\n명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.\n### Instruction(명령어):동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.\n### Response(응답):']

## 4. 모델 호출

### 4.1 모델 학습을 위한 기본 확인사항 내용

* 모델 파라미터 정보 확인

In [17]:
def print_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in model.parameters())
    print(
        f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}"
    )

* GPU 분산학습 설정 사용 유무 점검

In [18]:
ddp = False

def get_device_map():
    print(f"num_gpus: {torch.cuda.device_count()}")
    world_size = int(os.environ.get("WORLD_SIZE", torch.cuda.device_count()))
    print(f"world_size: {world_size}")
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        GRADIENT_ACCUMULATION_STEPS = TRAIN_BATCH_SIZE // world_size
        if GRADIENT_ACCUMULATION_STEPS == 0:
            GRADIENT_ACCUMULATION_STEPS = 1
        print(f"ddp is on - gradient_accumulation_steps: {GRADIENT_ACCUMULATION_STEPS}")
    else:
        device_map = "auto"
        print("ddp is off")

    return device_map

## 4.2 모델 Load

* 양자화 설정

In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=LOAD_IN_4BIT,
    bnb_4bit_use_double_quant=BNB_4BIT_USE_DOUBLE_QUANT,
    bnb_4xqbit_quant_type=BNB_4BIT_QUANT_TYPE,
    bnb_4bit_computxe_dtype=BNB_4BIT_COMPUTE_DTYPE
)

* 모델 읽어오기

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_OR_PATH,
    quantization_config=bnb_config,
    device_map=get_device_map(),
    cache_dir=CACHE_DIR,
    token=HUGGINGFACE_TOKEN
)

print_trainable_parameters(model)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

num_gpus: 1
world_size: 1
ddp is off


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


## 4.3 LoRA Adapter 적용

In [21]:
model = PeftModel.from_pretrained(
    model,
    CHECKPOINT_DIR,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)
model = model.merge_and_unload()
print_trainable_parameters(model)


if not ddp and torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True
    print("not ddp - trying its own DataParallelism")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


trainable params: 0 || all params: 3500412928 || trainable%: 0.0


In [22]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

### 4.3 Tokenizer 설정

* 모델 tokenizer를 통해 사용할 데이터셋을 분석하여 입력 데이터의 길이 분포를 파악

In [23]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, cache_dir=CACHE_DIR, use_add_token=True)
tokenizer.pad_token = tokenizer.eos_token

## 5. 모델을 통한 추론

### 5.1 추론

In [24]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer)

In [25]:
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.9,
    top_k=50,
    max_new_tokens=MAX_LENGTH,
    early_stopping=True,
    do_sample=True,
    repetition_penalty=1.1
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [26]:
streamer = TextStreamer(tokenizer)

def gen(instruction, input=None):
    # query = f"### instruction: {x}\n\n### Response: "
    query = PROMPT_DICT['prompt_no_input'] % instruction
    if input:
        query = PROMPT_DICT['prompt_input'] % (instruction, input)
    generated_id = model.generate(
        **tokenizer(
            query,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    generated_text = tokenizer.decode(generated_id[0], skip_special_tokens=True)

    response_start_idx = generated_text.find("### Response(응답):") + len("### Response(응답):")
    response = generated_text[response_start_idx:].strip()

    return response

## 일반 상식 예제

In [27]:
gen("""
파이썬 공부를 하고 싶습니다. 파이썬을 처음 공부해봅니다. 어떻게 공부를 시작하면 좋을까요?
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
파이썬 공부를 하고 싶습니다. 파이썬을 처음 공부해봅니다. 어떻게 공부를 시작하면 좋을까요?

### 

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Response(응답):파이썬에 대한 기본적인 지식은 약간의 웹 개발 경험과 함께 쉽게 익히실 수 있습니다. 그러나 완전히 파이썬을 익힐 때는 직접 문제를 해결하는 방법을 사용하는 것이 가장 좋습니다. 즉, 문제를 해결하는 동안 자신이 무엇을 어떻게 해야하는지 생각하는 것이 중요합니다. 이러한 방법들은 여러 가지가 있습니다. 예를 들어, 문제를 해결하는 동안 코드를 작성하는 것은 좋은 방법입니다. 또한, 문제를 해결하는 동안 다른 사람들이 해결하는 방법을 보는 것도 유용합니다. 이러한 방법들은 모든 문제를 해결하는 것보다는 일부 문제를 해결하는 것을 위주합니다. 따라서 파이썬을 공부하는 동안 문제를 해결하는 방법을 찾아내는 것이 중요합니다.</s>


'파이썬에 대한 기본적인 지식은 약간의 웹 개발 경험과 함께 쉽게 익히실 수 있습니다. 그러나 완전히 파이썬을 익힐 때는 직접 문제를 해결하는 방법을 사용하는 것이 가장 좋습니다. 즉, 문제를 해결하는 동안 자신이 무엇을 어떻게 해야하는지 생각하는 것이 중요합니다. 이러한 방법들은 여러 가지가 있습니다. 예를 들어, 문제를 해결하는 동안 코드를 작성하는 것은 좋은 방법입니다. 또한, 문제를 해결하는 동안 다른 사람들이 해결하는 방법을 보는 것도 유용합니다. 이러한 방법들은 모든 문제를 해결하는 것보다는 일부 문제를 해결하는 것을 위주합니다. 따라서 파이썬을 공부하는 동안 문제를 해결하는 방법을 찾아내는 것이 중요합니다.'

In [30]:
gen(
    instruction='컴퓨터 공학과의 일반적인 커리큘럼은 어떻게 되나요?',
    input='답변을 할 때, 2개의 문장으로 작성해주세요.'
)

<s> 아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):컴퓨터 공학과의 일반적인 커리큘럼은 어떻게 되나요?
### Input(입력):답변을 할 때, 2개의 문장으로 작성해주세요.
### Response(응답):컴퓨터 공학과에서는 기초 수업부터 전공 수업까지 진행됩니다. 기초 수업에서는 C/C++를 비롯한 프로그래밍 언어를 배우며, 전공 수업에서는 데이터베이스, 소프트웨어 시스템, 운영체제 등 여러 전공 분야를 선택할 수 있습니다.</s>


'컴퓨터 공학과에서는 기초 수업부터 전공 수업까지 진행됩니다. 기초 수업에서는 C/C++를 비롯한 프로그래밍 언어를 배우며, 전공 수업에서는 데이터베이스, 소프트웨어 시스템, 운영체제 등 여러 전공 분야를 선택할 수 있습니다.'

## 단순 코드 계산 예제

In [33]:
gen("""
파이썬을 통해 1부터 10까지의 총합을 구하는 코드를 작성해주세요
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
파이썬을 통해 1부터 10까지의 총합을 구하는 코드를 작성해주세요

### Response(응답):```python
sum = 0
for i in range(1, 11):
   sum += i
print("The sum of numbers from 1 to 10 is:", sum)
```</s>


'```python\nsum = 0\nfor i in range(1, 11):\n    sum += i\nprint("The sum of numbers from 1 to 10 is:", sum)\n```'

In [34]:
gen("""
자바 언어를 통해 1부터 15까지 곱셈하는 코드를 작성해주세요
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
자바 언어를 통해 1부터 15까지 곱셈하는 코드를 작성해주세요

### Response(응답):```java
int i = 1;
for (i = 2; i <= 15; i++) {
   System.out.println("" + i);
}
```</s>


'```java\nint i = 1;\nfor (i = 2; i <= 15; i++) {\n    System.out.println("" + i);\n}\n```'

## 요약 예제

In [37]:
gen("""
아래와 같은 기사가 있습니다. 기사의 핵심 내용을 추려서 간단하게 요약해주세요.
""",
"""
토트넘(잉글랜드)의 3연승을 이끈 '캡틴' 손흥민(31)이 맨 오브 더 매치(MOM)에 선정됐다. 
손흥민은 크리스마스 이브인 24일(한국 시각) 영국 런던의 토트넘 홋스퍼 스타디움에서 열린 에버턴과 2023-2024시즌 잉글랜드 프리미어리그(EPL) 18라운드 홈 경기에서 팀의 두 번째 골을 터뜨렸다. 
토트넘은 손흥민의 득점에 힘입어 2 대 1 승리를 거뒀다. 
이로써 토트넘은 3연승 행진을 이어갔고, 11승 3무 4패 승점 36을 기록했다. 한 경기를 덜 치른 맨체스터 시티(승점 34)를 제치고 4위로 올라섰다.
손흥민은 이날도 왼쪽 측면 공격수로 나섰다. 지난 16라운드 뉴캐슬전(1골 2도움)부터 왼쪽 측면에서 최고 윙어의 면모를 유감없이 발휘하고 있다. 
17라운드 노팅엄 포레스트전에서는 공격 포인트를 올리지 못했으나, 이날 2경기 만에 다시 득점포를 가동했다.
리그 11호 골을 터뜨린 손흥민은 무함마드 살라흐(리버풀), 재러드 보웬(웨스트햄)과 나란히 득점 공동 3위에 올랐다. 
1위는 14골의 엘링 홀란(맨체스터 시티), 2위는 12골의 도미닉 솔란케(본머스)다.
또 손흥민은 리그 반환점을 1경기 남겨둔 시점에서 벌써 지난 시즌 득점 기록을 넘어섰다. 
스포츠 탈장 부상 여파로 고전했던 지난 시즌에는 10골 6도움을 기록했다. 
도움 4개를 기록 중인 그는 지난 시즌 공격 포인트 기록 돌파도 눈앞에 두고 있다.
EPL 통산 득점 랭킹에서는 아스널의 레전드 이안 라이트(113골)을 넘어섰다. 
114골로 단독 23위에 오른 손흥민은 120골로 공동 21위인 라힘 스털링(첼시), 스티븐 제라드를 6골 차로 쫓고 있다.
손흥민은 경기 후 EPL 사무국이 22947명의 팬을 상대로 진행한 투표에서 67.7%의 압도적인 지지를 받아 MOM에 오르는 영예를 안았다. 
팀 동료인 굴리엘모 비카리오(15.1%), 페드로 포로(7.8%) 등을 크게 따돌렸다.
풀타임을 뛴 손흥민은 1골을 포함해 슈팅 2회, 패스 성공률 71%(24/34), 기회 창출 1회, 볼 터치 56회, 드리블 성공 43%(3/7) 등을 기록했다. 
축구 통계 매체 '풋몹'은 손흥민에게 팀 내 4번째로 높은 평점 7.8을 부여했다.
""")

매<s> 아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):
아래와 같은 기사가 있습니다. 기사의 핵심 내용을 추려서 간단하게 요약해주세요.

### Input(입력):
토트넘(잉글랜드)의 3연승을 이끈 '캡틴' 손흥민(31)이 맨 오브 더 매치(MOM)에 선정됐다. 
손흥민은 크리스마스 이브인 24일(한국 시각) 영국 런던의 토트넘 홋스퍼 스타디움에서 열린 에버턴과 2023-2024시즌 잉글랜드 프리미어리그(EPL) 18라운드 홈 경기에서 팀의 두 번째 골을 터뜨렸다. 
토트넘은 손흥민의 득점에 힘입어 2 대 1 승리를 거뒀다. 
이로써 토트넘은 3연승 행진을 이어갔고, 11승 3무 4패 승점 36을 기록했다. 한 경기를 덜 치른 맨체스터 시티(승점 34)를 제치고 4위로 올라섰다.
손흥민은 이날도 왼쪽 측면 공격수로 나섰다. 지난 16라운드 뉴캐슬전(1골 2도움)부터 왼쪽 측면에서 최고 윙어의 면모를 유감없이 발휘하고 있다. 
17라운드 노팅엄 포레스트전에서는 공격 포인트를 올리지 못했으나, 이날 2경기 만에 다시 득점포를 가동했다.
리그 11호 골을 터뜨린 손흥민은 무함마드 살라흐(리버풀), 재러드 보웬(웨스트햄)과 나란히 득점 공동 3위에 올랐다. 
1위는 14골의 엘링 홀란(맨체스터 시티), 2위는 12골의 도미닉 솔란케(본머스)다.
또 손흥민은 리그 반환점을 1경기 남겨둔 시점에서 벌써 지난 시즌 득점 기록을 넘어섰다. 
스포츠 탈장 부상 여파로 고전했던 지난 시즌에는 10골 6도움을 기록했다. 
도움 4개를 기록 중인 그는 지난 시즌 공격 포인트 기록 돌파도 눈앞에 두고 있다.
EPL 통산 득점 랭킹에서는 아스널의 레전드 이안 라이트(113골)을 넘어섰다. 
114골로 단독 23위에 오른 손흥민은 120골로 공동 21위인 라힘 스털링(첼시), 스티븐 제라드를 6골 차로 쫓고 있다.
손흥민은 경기 후 EPL

'손흥민이 맨 오브 더 매치에 선정된 것은 자신의 좋은 활약에 따른 것이며, 손흥민은 최근 3연승을 이끌었다. 손흥민은 왼쪽 측면 공격수로 나섰다. 손흥민은 맨체스터 시티와 본머스 등 강력한 라이벌들과 경합하며 많은 경기를 뛰고 있다.'

* 요약을 할때 모델이 2문장으로 잘 요약했지만, 두 문장의 답변을 얻기 위해 instruction에 별도의 설명을 추가하였음
* 많은 경우에 있어 n문장으로 요약을 해달라고 하면, n문장으로 답변을 해주지는 않음.
    * koalpaca 데이터만 학습시켰기 때문이며, 다음의 논리를 이해하기 위한 데이터셋을 추가하여 학습시킨다면 잘 답변을 해줄 것이라 생각이 됨

In [42]:
gen(
    instruction='아래와 같은 문장들이 있습니다. 답변을 할때 이 문장들을 2개의 문장으로 요약해주세요. 문장이 2개라는 것은 개행문자가 2개인 것을 의미합니다.',
    input=
"""
FLAN (Fine-tuned LAnguage Net) 모델은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다.
이 모델의 핵심 아이디어는 다양한 NLP 과제를 자연어 지시사항 형태로 변형하여 이러한 과제들을 풀도록 fine-tuning하는 것입니다.
이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning 됩니다​.

FLAN의 연구 결과에 따르면, 이 모델은 zero-shot 시나리오에서 GPT-3보다 우수한 결과를 보였으며, 많은 task에서는 supervised model과 비슷한 성능을 달성했습니다.
특히 자연어 추론(NLI)과 질문응답(QA) 작업에서 효과적이었습니다.
Google Research Blog에서는 FLAN이 언어 모델을 사용하여 특정 실제 과제에 대한 지식을 어떻게 풀어내는지에 대해 설명합니다.
전통적으로는 레이블이 붙은 데이터셋을 이용해 fine-tuning하는 방법이 많이 사용되었지만, FLAN은 다양한 종류의 지시사항에 대해 모델을 fine-tuning함으로써, 특정 과제가 아닌 일반적인 NLP 과제들을 해결할 수 있게 만듭니다.
""")

�<s> 아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):아래와 같은 문장들이 있습니다. 답변을 할때 이 문장들을 2개의 문장으로 요약해주세요. 문장이 2개라는 것은 개행문자가 2개인 것을 의미합니다.
### Input(입력):
FLAN (Fine-tuned LAnguage Net) 모델은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다.
이 모델의 핵심 아이디어는 다양한 NLP 과제를 자연어 지시사항 형태로 변형하여 이러한 과제들을 풀도록 fine-tuning하는 것입니다.
이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning 됩니다​.

FLAN의 연구 결과에 따르면, 이 모델은 zero-shot 시나리오에서 GPT-3보다 우수한 결과를 보였으며, 많은 task에서는 supervised model과 비슷한 성능을 달성했습니다.
특히 자연어 추론(NLI)과 질문응답(QA) 작업에서 효과적이었습니다.
Google Research Blog에서는 FLAN이 언어 모델을 사용하여 특정 실제 과제에 대한 지식을 어떻게 풀어내는지에 대해 설명합니다.
전통적으로는 레이블이 붙은 데이터셋을 이용해 fine-tuning하는 방법이 많이 사용되었지만, FLAN은 다양한 종류의 지시사항에 대해 모델을 fine-tuning함으로써, 특정 과제가 아닌 일반적인 NLP 과제들을 해결할 수 있게 만듭니다.

### Response(응답):FLAN은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다. 이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning

"FLAN은 자연어 처리(NLP) 과제들을 해결하기 위해 'instruction tuning'이라는 기법을 사용하는 언어 모델입니다. 이를 통해 FLAN 모델은 번역, 상식 추론, 감정 분류 등을 포함한 다양한 NLP 과제를 수행할 수 있도록 fine-tuning 됩니다."

## One-shot 예제

* 학습되지 않아 LLM에 지식이 존재하지 않음

In [43]:
gen("""
솔트룩스 회사의 루시아 솔루션에 대해서 설명해주세요.
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
솔트룩스 회사의 루시아 솔루션에 대해서 설명해주세요.

### Response(응답):Lucid Solutions은 미국 워싱턴 D.C. 지역에 위치한 소프트웨어 개발 회사로, 2014년 설립되었습니다. 회사는 정부 기관과 비영리 단체들에게 전자 문서 관리 시스템인 Lucid Viewer를 제공합니다. 이 시스템은 정부 기관과 비영리 단체들에게 전자 문서 관리 시스템인 Lucid Viewer를 제공합니다.</s>


'Lucid Solutions은 미국 워싱턴 D.C. 지역에 위치한 소프트웨어 개발 회사로, 2014년 설립되었습니다. 회사는 정부 기관과 비영리 단체들에게 전자 문서 관리 시스템인 Lucid Viewer를 제공합니다. 이 시스템은 정부 기관과 비영리 단체들에게 전자 문서 관리 시스템인 Lucid Viewer를 제공합니다.'

* 지식 하나를 주입

In [45]:
gen("""
질문 : 솔트룩스 회사의 루시아 솔루션에 대해서 설명해주세요. 솔트룩스 및 루시아에 대한 정보를 External Generated Knowledge로 드립니다.

```
인공지능 기업 솔트룩스(대표 이경일)는 ‘LUXIA Is All You Need–생성 AI 시대, 모든 것이 달라집니다'를 주제로 7일, 서울 코엑스 오디토리움에서 AI 컨퍼런스 'SAC 2023'을 개최했다.

이날 행사에서는 솔트룩스가 올해 초 금융권 컨퍼런스를 통해 처음 공개한 초거대 언어모델(LLM) ‘루시아(LUXIA)’와 그 생태계가 본격적으로 소개됐다. 온오프라인 하이브리드 형태로 개최된 이번 행사의 사전 참가 신청자는 약 2,700명에 달했다.

솔트룩스 이경일 대표는 키노트를 통해 도서 420만 권 분량을 학습한 자체 GPT 모델에 실시간 정보와 전문 지식을 활용해 환각 현상을 획기적으로 감소시킨 루시아GPT와 다양한 노코드 도구들을 선보였다. 또한 “처음 설립되었을 때 세상 모든 사람이 자유롭게 지식 소통하는 세상을 만들겠다는 미션을 가지고 있었던 솔트룩스는, 이제 오직 사람을 위한 인공지능 사람만을 위한 루시아GPT를 만들고 이를 통해 또 다른 성장을 시작하고자 한다”라고 말했다.  

다양한 환경에 맞춤형으로 도입 가능한 똑똑한 루시아(LUXIA)

루시아는 AI 데이터 구축 관련 정부 사업뿐 아니라 특허청, 행정안전부 등 다양한 분야의 사업을 수행하며 솔트룩스가 축적해 온 한글 데이터를 약 1TB 이상 학습했다. 이에 데이터 저작권 이슈를 최소화할 뿐 아니라 법률, 특허, 금융, 교육 등 각 전문 분야에 최적화된 맞춤형 언어모델을 빠르고 안전하게 구축할 수 있다. 

챗GPT 등 생성 AI의 고질적인 문제로 지적된 환각 현상을 최소화하기 위해서 ‘지식그래프(Knowledge Graph)를 활용한 사실/지식 그라운딩(Factual Grounding)’과 ‘검색 증강 생성(RAG·Retrieval-Augmented Generation)’이라는 2가지 접근법을 연계했다.

솔트룩스 김재은 랩장은 자사 지식그래프와 연계하여 자체 연구·개발한 인스트럭트 지식 학습(IKL·Instruct Knowledge Learning)을 통해 오픈AI의 ‘GPT-3.5’ 및 메타의 ‘라마(Llama)2’와 대비했을 때, 한국어 할루시네이션 자체 평가에서 대략 40% 더 우수한 성능을 확인할 수 있었다고 밝혔다. 

출처 : 인공지능신문(https://www.aitimes.kr)
```
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
질문 : 솔트룩스 회사의 루시아 솔루션에 대해서 설명해주세요. 솔트룩스 및 루시아에 대한 정보를 External Generated Knowledge로 드립니다.

```
인공지능 기업 솔트룩스(대표 이경일)는 ‘LUXIA Is All You Need–생성 AI 시대, 모든 것이 달라집니다'를 주제로 7일, 서울 코엑스 오디토리움에서 AI 컨퍼런스 'SAC 2023'을 개최했다.

이날 행사에서는 솔트룩스가 올해 초 금융권 컨퍼런스를 통해 처음 공개한 초거대 언어모델(LLM) ‘루시아(LUXIA)’와 그 생태계가 본격적으로 소개됐다. 온오프라인 하이브리드 형태로 개최된 이번 행사의 사전 참가 신청자는 약 2,700명에 달했다.

솔트룩스 이경일 대표는 키노트를 통해 도서 420만 권 분량을 학습한 자체 GPT 모델에 실시간 정보와 전문 지식을 활용해 환각 현상을 획기적으로 감소시킨 루시아GPT와 다양한 노코드 도구들을 선보였다. 또한 “처음 설립되었을 때 세상 모든 사람이 자유롭게 지식 소통하는 세상을 만들겠다는 미션을 가지고 있었던 솔트룩스는, 이제 오직 사람을 위한 인공지능 사람만을 위한 루시아GPT를 만들고 이를 통해 또 다른 성장을 시작하고자 한다”라고 말했다.  

다양한 환경에 맞춤형으로 도입 가능한 똑똑한 루시아(LUXIA)

루시아는 AI 데이터 구축 관련 정부 사업뿐 아니라 특허청, 행정안전부 등 다양한 분야의 사업을 수행하며 솔트룩스가 축적해 온 한글 데이터를 약 1TB 이상 학습했다. 이에 데이터 저작권 이슈를 최소화할 뿐 아니라 법률, 특허, 금융, 교육 등 각 전문 분야에 최적화된 맞춤형 언어모델을 빠르고 안전하게 구축할 수 있다. 

챗GPT 등 생성 AI의 고질적인 문제로 지적된 환각 현상을 최소화하기 위해서 ‘지식그래프(Knowledge Graph)를 활용한 사실/지식 그라운딩(Factua

'솔트룩스는 초거대 언어모델 LLM을 개발하고 있다. 이 언어모델은 생성 AI 시대에 적합한 모델이라고 주장한다. 솔트룩스는 이 언어모델을 통해 데이터 저작권 문제를 해결하고, 법률, 특허, 금융, 교육 등 다양한 분야에 최적화된 맞춤형 언어모델을 빠르고 안전하게 구축할 수 있다고 주장한다.'

* 학습되지 않아 LLM에 지식이 존재하지 않음

In [46]:
gen("""
슈카월드가 뭐에요
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
슈카월드가 뭐에요

### Response(응답):슈카월드는 플랫폼 기반의 모바일 게임입니다.</s>


'슈카월드는 플랫폼 기반의 모바일 게임입니다.'

In [47]:
gen("""
질문 : 슈카월드가 뭐에요. 슈카월드 대한 정보를 External Generated Knowledge로 드립니다.
슈카(본명: 전석재, 1979년~)는 대한민국의 유튜버이다. 증권 펀드매니저 출신으로, 경제 전문 1인 방송 슈카월드를 운영하고있다.
매주 일요일 저녁, 슈카월드 유튜브 채널에서 라이브 방송을 진행한다
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
질문 : 슈카월드가 뭐에요. 슈카월드 대한 정보를 External Generated Knowledge로 드립니다.
슈카(본명: 전석재, 1979년~)는 대한민국의 유튜버이다. 증권 펀드매니저 출신으로, 경제 전문 1인 방송 슈카월드를 운영하고있다.
매주 일요일 저녁, 슈카월드 유튜브 채널에서 라이브 방송을 진행한다

### Response(응답):슈카월드는 경제 전문 1인 방송으로, 생산자들과 소비자들의 상호작용을 중심으로 경제 현장을 알리고 살피기 위해 만들어진 채널입니다. 슈카월드는 2015년 4월 30일에 개설되었으며, 현재 160만 명의 구독자를 보유하고 있습니다.</s>


'슈카월드는 경제 전문 1인 방송으로, 생산자들과 소비자들의 상호작용을 중심으로 경제 현장을 알리고 살피기 위해 만들어진 채널입니다. 슈카월드는 2015년 4월 30일에 개설되었으며, 현재 160만 명의 구독자를 보유하고 있습니다.'

## CoT

* 수학적 연산을 LLM이 제대로 하지 못함

In [48]:
gen("""
한 반에 30명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까?
""")

<s> 아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):
한 반에 30명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까?

### Response(응답):15명</s>


'15명'

* CoT의 Few-shot 예제를 추가하였음에도 제대로 답변을 못함
    * KoAlpaca 데이터만 학습을 하였기에 LLM이 수학적인 사고를 하지 못함
* 다음번에 학습시킬 때에는 CoT 데이터셋을 추가하고 실험을 해서 CoT 문제를 해결해보자!

In [49]:
gen(
    instruction = '한 반에는 소년과 소녀로 이루어져 있으며, 총 60명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까? input의 예제를 보고 논리적으로 분해하고 생각을 하여 올바른 답변을 해주세요',
    input=
"""
예제1)
먼저, 반 전체 학생 수인 24명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 4분의 3에 해당합니다.
24명을 4로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 24를 4로 나누면 6명입니다.
이제 이 수를 3배하면 소녀의 수를 알 수 있습니다. 6명의 3배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 24명에서 18명을 빼면 6명이 남습니다.
따라서 이 반에는 소년이 6명 있습니다.
Response: 6명

예제2)
먼저, 반 전체 학생 수인 28명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 3분의 2에 해당합니다.
28명을 3으로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 28을 3으로 나누면 9.33, 즉 약 9명입니다.
이제 이 수를 2배하면 소녀의 수를 알 수 있습니다. 9명의 2배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 28명에서 18명을 빼면 10명이 남습니다.
따라서 이 반에는 소년이 10명 있습니다.
Response: 10명

예제3)
먼저, 도서관 전체 책의 수인 30권 중에서 과학 책의 비율을 계산해야 합니다. 과학 책은 전체의 5분의 4에 해당합니다.
30권을 5로 나누면 각 그룹에 몇 권이 있는지 알 수 있습니다. 30을 5로 나누면 6권입니다.
이제 이 수를 4배하면 과학 책의 수를 알 수 있습니다. 6권의 4배는 24권입니다. 그러므로 과학 책은 24권입니다.
전체 책의 수에서 과학 책의 수를 빼면 문학 책의 수를 알 수 있습니다. 30권에서 24권을 빼면 6권이 남습니다.
따라서 도서관에는 문학 책이 6권 있습니다.
Response : 6권

그렇다면 instruction의 정답은??
""")

<s> 아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.
명령어와 입력을 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.

### Instruction(명령어):한 반에는 소년과 소녀로 이루어져 있으며, 총 60명의 학생이 있습니다. 그 중 3분의 2가 소녀입니다. 소년은 몇 명입니까? input의 예제를 보고 논리적으로 분해하고 생각을 하여 올바른 답변을 해주세요
### Input(입력):
예제1)
먼저, 반 전체 학생 수인 24명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 4분의 3에 해당합니다.
24명을 4로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 24를 4로 나누면 6명입니다.
이제 이 수를 3배하면 소녀의 수를 알 수 있습니다. 6명의 3배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 24명에서 18명을 빼면 6명이 남습니다.
따라서 이 반에는 소년이 6명 있습니다.
Response: 6명

예제2)
먼저, 반 전체 학생 수인 28명 중에서 소녀의 비율을 계산해야 합니다. 소녀는 전체의 3분의 2에 해당합니다.
28명을 3으로 나누면 각 그룹에 몇 명이 있는지 알 수 있습니다. 28을 3으로 나누면 9.33, 즉 약 9명입니다.
이제 이 수를 2배하면 소녀의 수를 알 수 있습니다. 9명의 2배는 18명입니다. 그러므로 소녀는 18명입니다.
전체 학생 수에서 소녀의 수를 빼면 소년의 수를 알 수 있습니다. 28명에서 18명을 빼면 10명이 남습니다.
따라서 이 반에는 소년이 10명 있습니다.
Response: 10명

예제3)
먼저, 도서관 전체 책의 수인 30권 중에서 과학 책의 비율을 계산해야 합니다. 과학 책은 전체의 5분의 4에 해당합니다.
30권을 5로 나누면 각 그룹에 몇 권이 있는지 알 수 있습니다. 30을 5로 나누면 6권입니다.
이제 이 수를 4배하면 과학 책의 수를 알 수 있습니다. 6권의 4배는 24권입

'소년은 6명, 소녀는 18명입니다.'

## 테스트 데이터셋 검증(held-out 데이터는 아님)

In [24]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_k=1,
    num_beams=3,
    do_sample=True,
)

In [25]:
generated_texts = []

for prompt in formatted_prompts[:10]:
    model_input = tokenizer(prompt,  padding=True, truncation=False, max_length=MAX_LENGTH, return_tensors='pt')

    generated_id = model.generate(
        **model_input,
        max_new_tokens=MAX_LENGTH,
        early_stopping=True,
        generation_config=generation_config,
        output_scores=True,
        return_dict_in_generate=False,
    )

    generated_text = tokenizer.decode(generated_id[0], skip_special_tokens=True)
    print(generated_text)
    print('-' * 100)
    generated_texts.append(generated_text)

final_responses = []
for output in generated_texts:
    response_start_idx = output.find("### Response(응답):") + len("### Response(응답):")
    response = output[response_start_idx:].strip()
    final_responses.append(response)

final_responses

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):고객 서비스 봇에 가장 적합한 어조를 파악하세요.
### Response(응답):"저희 봇은 고객 서비스를 위해 설계되었습니다. 저희 봇은 고객의 요구에 따라 적절한 대응을 할 수 있습니다."
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):동물 멸종 사건에 대한 뉴스 기사의 헤드라인을 생성합니다.
### Response(응답):"새로운 멸종 사건 발견"
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 꼼꼼하고 차분하게 응답을 작성하세요.
### Instruction(명령어):예산 수립을 위한 계획 만들기
### Response(응답):예산 수립을 위한 계획을 만들기 위해서는 예산 수준과 예산 필요성을 파악하고 예산 계획을 작성해야 합니다. 이를 위해서는 예산 수준을 파악하는 방법과 예산 필요성을 파악하는 방법을 알아야 합니다. 예산 수준을 파악하는 방법은 예산 계획을 작성하는 방법과 같습니다. 예산 필요성을 파악하는 방법은 예산 계획을 작성하는 방법과 같습니다. 이를 위해서는 예산 계획을 작성하는 방법과 같은 방법을 사용해야 합니다.
----------------------------------------------------------------------------------------------------
아래는 작업을 설명하는 명령어입니다.
명령어를 깊게 이해하고 

['"저희 봇은 고객 서비스를 위해 설계되었습니다. 저희 봇은 고객의 요구에 따라 적절한 대응을 할 수 있습니다."',
 '"새로운 멸종 사건 발견"',
 '예산 수립을 위한 계획을 만들기 위해서는 예산 수준과 예산 필요성을 파악하고 예산 계획을 작성해야 합니다. 이를 위해서는 예산 수준을 파악하는 방법과 예산 필요성을 파악하는 방법을 알아야 합니다. 예산 수준을 파악하는 방법은 예산 계획을 작성하는 방법과 같습니다. 예산 필요성을 파악하는 방법은 예산 계획을 작성하는 방법과 같습니다. 이를 위해서는 예산 계획을 작성하는 방법과 같은 방법을 사용해야 합니다.',
 '"Dear John"은 작가 존 디페이의 작품에서 유래된 용어입니다. 존 디페이는 1960년대 초반 미국 작가로 유명했습니다. 그는 여러 가지 작품을 남겼는데, 그 중 하나가 1963년 출간된 소설 "The Sun is Burning"입니다. 이 소설은 작가 자신의 이별 경험을 바탕으로 쓰여졌습니다. 이 소설에서 존 디페이는 자신의 이별 경험을 작품에 담아 출판했습니다. 이 소설에서 존 디페이는 자신의 이별 경험을 "Dear John"이라는 표현으로 묘사했습니다. 이 표현은 이후 미국 문학계에서 유명해졌으며, 이후 이별 편지나 절교장을 의미하는 표현으로 사용되게 되었습니다. 이 용어는 현재까지도 미국 문학계에서 사용되고 있습니다.',
 '```\npublic static void main(String[] args) {\n    for (int i = 0; i < 10; i++) {\n        System.out.println(fibonacci(i));\n    }\n}\n\npublic static int fibonacci(int n) {\n    if (n == 0) {\n        return 0;\n    } else if (n == 1) {\n        return 1;\n    } else {\n        return fibonacci(n - 1) + fibonacci(n 

### 5.2 추론한 내용 다음 번에 재활용하기 위해 pickle 파일로 직렬화

In [47]:
import pickle
with open(GENERATION_DATA_LIST_PICKLE_PATH, 'wb') as file:
    pickle.dump(final_responses, file)

### 5.3 저장되어져있는 추론 내용 pickle 파일로 역직렬화를 통한 메모리 로드

In [7]:
import pickle

with open(GENERATION_DATA_LIST_PICKLE_PATH, 'rb') as file:
    generation_data_list = pickle.load(file)

## 6. 데이터 평가

In [41]:
reference_list = df_ko_alpaca_final_sampled['output'].tolist()
print(len(reference_list))
print(reference_list[0])

1000
고객 서비스에서 호칭과 제품 또는 서비스에 대한 지식을 바탕으로 열정적인 어조를 사용하는 것이 좋습니다.


In [49]:
from evaluate import load
import json

class LLMEvaluator:
    pass

In [ ]:
evaluator = LLMEvaluator()
result = evaluator.run_evaluate(
    predictions=generation_data_list, 
    origins=reference_list[:10],
    use_save_as_file=True,
)